# Tarea 9.10

Ejercicio 4.6.10 del libro de análisis numérico de Burden.

El estudio de la difracción de la luz en una apertura rectangular involucra las integrales de Fresnel

$$
c(t) = \int_0^t dw\,\cos\frac{\pi}{2}w^2\,,\quad s(t) = \int_0^t dw\,\sin\frac{\pi}{2}w^2\,.
$$

Construya una tabla de integrales de Fresnel con una precisión de $10^{-4}$ para los valores $t = 0.1, 0.2, ..., 0.9, 1.0$.

In [28]:
import numpy as np
import matplotlib.pyplot as plt 

Definimos las funciones que están dentro de las ecuaciones de fresnel, las cuales serían  $C(w)$   y   $S(w)$

In [29]:
def C(w):
    return np.cos((np.pi/2)*w**2)
def S(w):
    return np.sin((np.pi/2)*w**2)

Ocupamos una función definida en las clases del profesor.

In [30]:
class MaxIterations(Exception):
    pass

def integral_adaptive(f, a, b, tol, N=100000):
    
    '''
    f: la función a integrar   (type:function)
    [a,b]: los limites de integracion   (type:float)
    tol: el valor de tolerancia    (type:float)
    N: cantidad maxima de iteraciones   (type:int) 
    '''
    
    #Variables iniciales
    approx = 0
    i = 0
    toli = [10*tol]
    ai = [a]
    hi = [(b - a)/2]
    fai = [f(a)]
    fbi = [f(b)]
    fci = [f(a + hi[i])]
    S0i = [hi[i]*(fai[i] + 4*fci[i] + fbi[i])/3]
    Li = [1]
    
    while i >= 0:
        
        fd = f(ai[i] + hi[i]/2)
        fe = f(ai[i] + 3*hi[i]/2)
        S1 = hi[i]*(fai[i] + 4*fd + fci[i])/6
        S2 = hi[i]*(fci[i] + 4*fe + fbi[i])/6
        ai_prec = ai[i]
        hi_prec = hi[i]
        fai_prec = fai[i]
        fbi_prec = fbi[i]
        fci_prec = fci[i]
        toli_prec = toli[i]
        S0i_prec = S0i[i]
        Li_prec = Li[i]
        
        i -= 1
        if abs(S1 + S2 - S0i_prec) < toli_prec:
            approx += S1 + S2
        else:
            if Li_prec >= N:
                raise MaxIterations("Alcanzado máximo número de iteraciones.")
            
            # Intervalo derecho
            i += 1
            if i >= len(ai): # A veces hay que ampliar la lista
                ai.append(ai_prec + hi_prec)
                fai.append(fci_prec)
                fci.append(fe)
                fbi.append(fbi_prec)
                hi.append(hi_prec/2)
                toli.append(toli_prec/2)
                S0i.append(S2)
                Li.append(Li_prec + 1)
            else:
                ai[i] = ai_prec + hi_prec
                fai[i] = fci_prec
                fci[i] = fe
                fbi[i] = fbi_prec
                hi[i] = hi_prec/2
                toli[i] = toli_prec/2
                S0i[i] = S2
                Li[i] = Li_prec + 1
                
            # Intervalo izquierdo
            i += 1
            if i >= len(ai):
                ai.append(ai_prec)
                fai.append(fai_prec)
                fci.append(fd)
                fbi.append(fci_prec)
                hi.append(hi[i-1])
                toli.append(toli[i-1])
                S0i.append(S1)
                Li.append(Li[i-1])
            else:
                ai[i] = ai_prec
                fai[i] = fai_prec
                fci[i] = fd
                fbi[i] = fci_prec
                hi[i] = hi[i-1]
                toli[i] = toli[i-1]
                S0i[i] = S1
                Li[i] = Li[i-1]
                
    return approx

Ya con la función definida que nos ejecuta la integración con el metodo adaptativo, tenemos que crear un np.array vacio para guardar los datos para los valores pedidos en la pregunta 

In [31]:
tabla_para_graficar = np.empty([0,2])
tabla_para_graficar

array([], shape=(0, 2), dtype=float64)

Definimos un loop que nos permite evaluar para cada $t$ para las dos funciones estudiadas $C$ y $S$

In [32]:
for i in np.arange (0,1.1,0.1):
    C_integrado=integral_adaptive(C,0,i,0.0001)
    S_integrado=integral_adaptive(S,0,i,0.0001)
    resultado_obtenidos= np.array([C_integrado,S_integrado])
    tabla_para_graficar=np.vstack([tabla_para_graficar,resultado_obtenidos])

In [33]:
tabla_para_graficar

array([[0.00000000e+00, 0.00000000e+00],
       [9.99975262e-02, 5.23589387e-04],
       [1.99920853e-01, 4.18758861e-03],
       [2.99399446e-01, 1.41166481e-02],
       [3.97474593e-01, 3.33568374e-02],
       [4.92327199e-01, 6.47203162e-02],
       [5.81060999e-01, 1.10498453e-01],
       [6.59649786e-01, 1.72129265e-01],
       [7.22844230e-01, 2.49325381e-01],
       [7.64971727e-01, 3.39746957e-01],
       [7.79880037e-01, 4.38245260e-01]])